In [29]:
import numpy as np
import math
from abc import ABC, abstractmethod
from numpy.random import default_rng
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
import pandas as pd




In [30]:
rng = default_rng(seed=0)


# Linear Regression

In [51]:
def regression(Xtrain, Xtest, ytrain, ytest) :
    reg = LinearRegression().fit(Xtrain, ytrain)
    
    return reg.coef_, reg.intercept_, reg.score(Xtest, ytest)

#  Data
We use auto-mpg dataset to work with

In [31]:
mpg = pd.read_csv('./auto-mpg.csv', na_values=['NA','?'])

In [32]:
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


## Pre Processing Dataset

In [33]:
print(mpg.columns)

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name'],
      dtype='object')


In [34]:
mpg['name']

0      chevrolet chevelle malibu
1              buick skylark 320
2             plymouth satellite
3                  amc rebel sst
4                    ford torino
                 ...            
393              ford mustang gl
394                    vw pickup
395                dodge rampage
396                  ford ranger
397                   chevy s-10
Name: name, Length: 398, dtype: object

In [35]:
cars = mpg['name']
mpg.drop('name',1,inplace=True)
## replacing NA values
med = mpg['horsepower'].median()
mpg['horsepower'] = mpg['horsepower'].fillna(med) 

/Users/baraa/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [37]:
## Selecting covariates columns
covariates = []
for x in mpg.columns:
    if x != 'mpg':
        covariates.append(x)

print(covariates)

['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']


In [38]:
## Getting data
X = mpg[covariates].values
y = mpg['mpg'].values

print('X shape', X.shape)
print('y shape', y.shape)

X shape (398, 7)
y shape (398,)


In [46]:
#split data into testing and training
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

## How to compare nested models ?
- Compare model result
- Run t-test on one additional covariate
- F-test
- Anova 

### Comparison using model $R_2$ score and RMSE

In [54]:
a,b,c = regression(X_train, X_test, y_train, y_test)
print('a :', a)
print('b :', b)
print('Regression R2 score :', c)

a : [-0.30958288  0.02106943 -0.02088593 -0.00650729  0.19986328  0.74059199
  1.62910052]
b : -19.47927266862647
Regression R2 score : 0.8139447246621095


In [59]:
from sklearn import metrics
# build the model
model = LinearRegression()  
model.fit(X_train, y_train)

#calculate the predictions of the linear regression model
y_pred = model.predict(X_test)

# Mean and RMSE
print('Covariates Used :', covariates)
print('Mean:', np.mean(y_test))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2 Score : ', model.score(X_test, y_test))

Covariates Used : ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']
Mean: 23.695
Root Mean Squared Error: 3.409213715841733
R2 Score :  0.8139447246621095


There is 7 covariates, we will drop one each time and see the model results

In [61]:
X_del = np.delete(X, 1,1)
print(X.shape)
print(X_del.shape)

(398, 7)
(398, 6)


In [64]:
n = len(covariates)

for i in range(n):
    
    # Dropping covariates[i]
    new_covariates = [ covariates[j] for j in range(n) if j!=i ]
    
    # New covariates Input
    new_X = np.delete(X, i, 1)
    new_X_train = np.delete(X_train, i, 1)
    new_X_test = np.delete(X_test, i, 1)
    
    #build model
    model = LinearRegression()
    model.fit(new_X_train, y_train)
    
    #calculate the predictions of the linear regression model
    new_y_pred = model.predict(new_X_test)

    # Mean and RMSE
    print('Covariates Dropped :', covariates[i])
    print('Mean:', np.mean(y_test))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, new_y_pred)))
    print('R2 Score : ', model.score(new_X_test, y_test))
    print('---------------------------------------------')

    
    
    

Covariates Dropped : cylinders
Mean: 23.695
Root Mean Squared Error: 3.424452971304196
R2 Score :  0.8122776652574544
---------------------------------------------
Covariates Dropped : displacement
Mean: 23.695
Root Mean Squared Error: 3.4287904131140543
R2 Score :  0.8118018226229958
---------------------------------------------
Covariates Dropped : horsepower
Mean: 23.695
Root Mean Squared Error: 3.386606964311643
R2 Score :  0.8164040365327735
---------------------------------------------
Covariates Dropped : weight
Mean: 23.695
Root Mean Squared Error: 3.9250218853567675
R2 Score :  0.7533860157568725
---------------------------------------------
Covariates Dropped : acceleration
Mean: 23.695
Root Mean Squared Error: 3.3575743228673374
R2 Score :  0.8195383997400633
---------------------------------------------
Covariates Dropped : year
Mean: 23.695
Root Mean Squared Error: 4.44483070879713
R2 Score :  0.6837402062075155
---------------------------------------------
Covariates Drop

### Conclusion

From the result we can see that the $R_2$ score drop significantly and RMSE gets higher when we drop either weight or year covariates, with the year having the stronger impact. We can conclude that these 2 covariates are the important ones. 

### Using Only weights


In [69]:
# New covariates Input
new_X = X[:,3]
new_X_train = X_train[:,3]
new_X_test = X_test[:,3]

#build model
model = LinearRegression()
model.fit(new_X_train.reshape(-1,1), y_train)

#calculate the predictions of the linear regression model
new_y_pred = model.predict(new_X_test.reshape(-1,1))

print('Covariates Used :', covariates[3])
print('Mean:', np.mean(y_test))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, new_y_pred)))
print('R2 Score : ', model.score(new_X_test.reshape(-1,1), y_test))


Covariates Used : weight
Mean: 23.695
Root Mean Squared Error: 4.19810204633154
R2 Score :  0.717876335343846


### Using Only years


In [71]:
# New covariates Input
new_X = X[:,5]
new_X_train = X_train[:,5]
new_X_test = X_test[:,5]

#build model
model = LinearRegression()
model.fit(new_X_train.reshape(-1,1), y_train)

#calculate the predictions of the linear regression model
new_y_pred = model.predict(new_X_test.reshape(-1,1))

print('Covariates Used :', covariates[5])
print('Mean:', np.mean(y_test))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, new_y_pred)))
print('R2 Score : ', model.score(new_X_test.reshape(-1,1), y_test))


Covariates Used : year
Mean: 23.695
Root Mean Squared Error: 6.403885661824743
R2 Score :  0.3435212499308733


### Using Only wights and years


In [76]:
# New covariates Input
new_X = X[:,[3,5]]
new_X_train = X_train[:,[3,5]]
new_X_test = X_test[:,[3,5]]

#build model
model = LinearRegression()
model.fit(new_X_train, y_train)

#calculate the predictions of the linear regression model
new_y_pred = model.predict(new_X_test)

print('Covariates Used :', [covariates[i] for i in [3,5]])
print('Mean:', np.mean(y_test))
print('Root Mean Squared Error Full Model: 3.409213715841733')
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, new_y_pred)))
print('R2 Score Full Model : 0.8139447246621095 ')
print('R2 Score : ', model.score(new_X_test, y_test))


Covariates Used : ['weight', 'year']
Mean: 23.695
Root Mean Squared Error Full Model: 3.409213715841733
Root Mean Squared Error: 3.3035255658048204
R2 Score Full Model : 0.8139447246621095 
R2 Score :  0.8253016196534047


### Conclusion
We have a better model using only the 'year' and 'weight' covariate. We still need to check that against other training/testing datasets